Here, I have tried to create a raw model for prediction of sentiment through this dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, average_precision_score, recall_score

import re

In [ ]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().any()

In [ ]:
stop_words=stopwords.words('english')
print(stop_words)

In [ ]:
stemmer = SnowballStemmer('english')
text_cleaning_re = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [ ]:
def preprocess(text, stem=False):
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)
df.review = df.review.apply(lambda x: preprocess(x))

In [ ]:
df.head()

In [ ]:
plt.figure(figsize = (20,20)) 
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(df.review[df.sentiment == 'positive']))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
from wordcloud import WordCloud

plt.figure(figsize = (20,20)) 
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(df.review[df.sentiment == 'negative']))
plt.imshow(wc , interpolation = 'bilinear')

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.review)

word_index = tokenizer.word_index
vocab_size=len(word_index)+1
print(vocab_size)

In [ ]:
y_values = dict([('positive' , 1), ('negative', 0) ])
y=df.sentiment.map(y_values)
df=df.drop('sentiment', axis=1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df,y,test_size=0.2, random_state=2021)

In [ ]:
x_train = pad_sequences(tokenizer.texts_to_sequences(x_train.review),
                        maxlen = 50)
x_test = pad_sequences(tokenizer.texts_to_sequences(x_test.review),
                       maxlen = 50)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

In [ ]:
model=LogisticRegression(solver='lbfgs')
model=OneVsRestClassifier(model)
model.fit(x_train,y_train)
Ypred=model.predict(x_test)

In [ ]:
def print_evaluation_scores(y_test, Ypred):
    print('Accuracy score: ', accuracy_score(y_test, Ypred))
    print('F1 score: ', f1_score(y_test, Ypred, average='micro'))
    print('Average precision score: ', average_precision_score(y_test, Ypred, average='micro'))
    print('Average recall score: ', recall_score(y_test, Ypred, average='micro'))

In [ ]:
print_evaluation_scores(y_test,Ypred)

In [ ]:
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

# Work under Progress